In [ ]:
import pandas as pd
import numpy as np

# input data 부분 필요 라이브러리
import kqc_custom
from dataclasses import dataclass, field, asdict, astuple
from typing import List, ClassVar
import ast  # 입력 문자 beta coef, integer 로 변환에 사용하는 부분

# Simulated Annealing 필요 라이브러리
from sa_basefunctions import get_aic, get_bic, get_mspe, get_bin, flip
from sa_SimulatedAnnealing import SimulatedAnnealing

# Genetic Algorithm 필요 라이브러리 

## Data Generator 구현 부분

*Data Generating Function 함수 설명*

$y = X \beta + \epsilon$

kqc_custom.generate_independent_sample(n_sample,n_features,beta_coef=[4,3,2,2],epsilon=4)  
kqc_custom.generate_dependent_sample(n_sample,n_features,beta_coef=[4,3,2,2],epsilon=4,correlation_parameter=1)

- n_samples : 데이터 샘플 수
- n_features : 데이터 피쳐 개수
- beta_coef : beta계수
- epsilon : 오차의 표준편차 정도
- correlation_parameter


In [ ]:
# inout parameter 가져오기
KQC_data_parameter = pd.read_excel('input_parameter.xlsx')

Example of input data generator funvtion's parameter input from EXCEL (.xlcs file)

    아래에 간단한 예시로 4개의 데이터의 경우를 예시로 들었습니다.

In [ ]:
KQC_data_parameter

In [ ]:
def read_excel2tuple():
    '''
        row 별로 data를 슬라이싱 해주는 Tuple로 변환
    '''
    df = pd.read_excel('input_parameter.xlsx')
    return list(df.itertuples(name=None))

In [ ]:
Tupled_df_KQC_data_parameter = list(KQC_data_parameter.itertuples(name=None))

In [ ]:
list(list(KQC_data_parameter.itertuples(name=None))[0])

EXCEL 의 dataframee 의 pandas로 받아서, 

In [ ]:
from typing import ClassVar
@dataclass
class data_generator:
    count_data : ClassVar[int] = 1  # 전치 처리위한 class variable

    # KQC custom file // data generator function's parameter
    n_samples : int                 # 데이터 샘플 수
    n_features : int                # 데이터 피쳐 개수
    beta_coef : float               # beta 계수
    epsilon : float                 # 오차의 표준편차 정도
    correlation_parameter : float   # 상관계수 ?

    # kqc data generator dependent / inpependent 선택 부분 
    is_independent : str            # is_independent
    data_name : str                 # 지정 데이터명

    def __post_init__(self):
        data_generator.count_data += 1  # class 생성 data counter
      
    # parameter 입력 받기 
    @classmethod
    def create(cls): 
        data = cls() 
        return data
    
    @classmethod
    def data_counter(cls):
        return [ 'data' + str(i) for i in range(cls.count_data)]

    #KQC_custom 의 data generator 로 부터 data 생성 class
    def data_gen(self):
        data_gened = []
        if self.is_independent == 'dependent':
            x, y = kqc_custom.generate_dependent_sample( self.n_samples,
                                                  self.n_features,
                                                  self.beta_coef,
                                                  self.epsilon,
                                                  self.correlation_parameter
            )
        else:
            x, y = kqc_custom.generate_independent_sample( self.n_samples,
                                                  self.n_features,
                                                  self.beta_coef,
                                                  self.epsilon,
                                                  self.correlation_parameter
            )
        data_gened.append(x)
        data_gened.append(y)
        return data_gened
# data_name : str 
# paeameter : List[float] = field(default_factory=list)

In [ ]:
ast.literal_eval(list(list(KQC_data_parameter.itertuples(name=None))[0])[4])

In [ ]:
# #새로운 data generator 함수 부분
# from dataclasses import make_dataclass
# make_dataclass(data_generator())

In [ ]:
# class data generator 로 부터, 입력 데이터 리스트의 수 만큼 리스트 생성 
def list_gen_from_class(df):
    generated_data_list = []
    Tupled_df_KQC_data_parameter = read_excel2tuple()
    for i in range(len(Tupled_df_KQC_data_parameter)):
        data_list_Row = list(list(df.itertuples(name=None))[i])
        generated_data_list.append(data_generator(n_samples= data_list_Row[2],
                                                n_features= data_list_Row[3],
                                                beta_coef= ast.literal_eval(data_list_Row[4]),
                                                epsilon= data_list_Row[5],
                                                correlation_parameter= data_list_Row[6],
                                                is_independent= data_list_Row[7],
                                                data_name=data_list_Row[1]
            )
        )
    return generated_data_list

다음은 generated data list 의 형태를 보여주는 예시입니다.  

아래의 예시와 같이 각 리스트에서 KQC_ independent / dependent data 의 생성되는  
x , y 의 값이 N(excel input 데이터수) by 2 의 size 리스트 입니다.  

In [ ]:
generated_data_list = list_gen_from_class(KQC_data_parameter)
#generated_data 의 length --> 클래스 변수에서 선언횟수 카운터로 선언했으므로 
# 임의의 첫번쨰 선언 instance 에서 클래스 매서드를 호출하면 조정없이 가능
data_key = generated_data_list[0].data_counter()

리스트와 동일하지만 검색에 유용하기 위해서, 데이터 명으로 검색을 하기 위해서 딕셔너리로 변환하는  부분을 작성해 보았습니다.  

In [ ]:
generated_data_dict = {}
for i in range(len(generated_data_list)):
    '''
        클래스에서 선언 인스턴스 개수로 자동 카운트 되는 이름명은 윗 코드
        dataframe 에서 선언한 '지정 데이터명'을 사용한다면 아래 코드를 사용하면 됩니다.
    '''
    generated_data_dict[data_key[i]] = generated_data_list[i]
    # list(KQC_data_parameter.loc[:,'지정 데이터명'])[i] = generated_data_list[i]

In [ ]:
generated_data_dict['data3']

In [ ]:
# 클래스 구조를 가짐으로서 data 이름을 검색한것으로 부터 parameter를 쉽게 확인할 수 있습니다.
generated_data_dict['data3'].__dict__

In [ ]:
# 저장된 클래스 인스턴스로 부터 데이터 생성하기 
generated_data_dict['data3'].data_gen()

## 알고리즘 적용 부분

### SA

In [ ]:
from sa_SimulatedAnnealing import SimulatedAnnealing as SA

In [ ]:
SA_input_parameter = pd.read_excel('sa_input_parameter.xlsx')
SA_input_parameter

In [ ]:
# SA input parameter 부분 
from sa_input_parameter_wrap import sa_read_excel2tuple, sa_parameter_wrap, sa_list_gen_from_class
from sa_results import sa_Results

다시 돌아와서 우리가 생성한 데이터는 아래이고 여기서 input data 에따른 data 이름값과  
이제 새로 생성한 sa_input_data 의 부분의 labeling 을 진행해줘야 됩니다. 

example -- > data 1 로 부터 sa_input_parameter data bla bla  

idea 1 : 각각의 데이터별로 excel 파일 그리고 sa_input_parameter 별로 sheet 만들기 

idea 2 : 그냥 폴더? --> 따로 os 생성자로 폴더 지정해 저장해 줄지 

naming은 'input data name ' + 'with' + 'sa_parameter_select' .dat로?

In [ ]:
# 출력을 저장할 데이터 클래스 생성 
@dataclass
class Results:
    #사용환경에 따른 수정요
    # default_information =  'N = 500, p = 20, beta_coef=[4, 3, 2, 2], epsilon =, covariance_parameter = 5, cooling_schedule = lundymee'#.format(eps)
    value : List[float] = field(default_factory=list)
    result : List[float] = field(default_factory=list)
    log : List[int] = field(default_factory=list)
    
    # pre-variable
    # def __post_init__(self):
        # self.information = self.default_information 
    # dict allocation
    def __getitem__(self,key):
        return getattr(self, key)
    def __setitem__(self,key,value):
        return setattr(self, key, value)

# class instance generator
def Results_gen(value, result, log):
    return Results(value, result, log)

In [ ]:
generated_sa_parameter_wrap_list = sa_list_gen_from_class(SA_input_parameter)

In [ ]:
sa_result_dict = {}
# 각 SA _INPUT_PATAMETER 의 FOR 문
for sa_parameter_data in generated_sa_parameter_wrap_list:
  #각 input 별 generated data set 
  for data_name, data in list(generated_data_dict.items()) :
    x, y = pd.DataFrame(data.data_gen()[0]), data.data_gen()[1]
    simulatedannealing = SA(x,y,
                            sa_parameter_data.lst,
                            sa_parameter_data.k,
                            sa_parameter_data.alpha,
                            sa_parameter_data.tau,
                            sa_parameter_data.objective
    )
    # data wrapper의 value로 부터 각각의 result 생성하기
    value, result, log = simulatedannealing.lundymee()
    new_name = data_name + '_with_' + sa_parameter_data.sa_input_parameter_name
    sa_result_dict[new_name] = sa_Results(value, result, log)

아래의 코드를 출력하면 최종결과를 얻을 수 있습니다.

* input_parameter 엑셀파일 에서 입력하는 파라미터별  KQC data generator 의 실험 데이트 set data1 ~ data4
* sa_input_parameter 엑셀파일에서 입력하는 파라미터별 sa_SimulatingAnnealing.py 의 변수별 wrap 설정 부분  sa_parameter1 ~ sa_parameter4

총 16개의 실험결과가 sa_result_dict 의 딕셔너리에 저장된것을 확인할 수 있습니다.  
데이터의 호출방법은 별도로 pickle로 저장할텐데요, 지금 주피터 노트북의 iPython 환경에서는 모든 value를 keep 하고 있기 때문에  
저장된 sa_result_dict을 바로 호출가능합니다.

1. 딕셔너리의 key (데이터 이름)
2. 이름을 통해서 각 data 별 저장된 Results class 에 접근 
3. 

In [ ]:
print(sa_result_dict.keys())
key_list = list(sa_result_dict.keys())

In [ ]:
# 임의의 key_list[4], 'data0_with_sa_parametter2' 의 결과
sa_results = sa_result_dict[key_list[4]]
sa_results.result

In [ ]:
sa_results.log

In [ ]:
sa_results.value

### GA



In [69]:
from ga_algorithm import GA
from ga_results import ga_Results_gen, ga_Results

SA 와 별개로 ga는 실험 데이터별로 사용할 매개변수의 조정이 별도로 없어서 4가지 종류의 input data로 부터
바로 GA 알고리즘을 실행합니다.  
아래는 생성한 data 4개의 딕셔너리 입니다.

In [72]:
print(generated_data_dict)
len(generated_data_dict.keys())

a,b = list(generated_data_dict.items())[1]
x = pd.DataFrame(b.data_gen()[0])
y = b.data_gen()[1]

print(x)
print(y)
# GA(x, y)

{'data0': data_generator(n_samples=500, n_features=10, beta_coef=[4, 3, 2, 2], epsilon=1, correlation_parameter=1, is_independent='independent', data_name='data1'), 'data1': data_generator(n_samples=500, n_features=10, beta_coef=[4, 3, 2, 2], epsilon=2, correlation_parameter=1, is_independent='independent', data_name='data2'), 'data2': data_generator(n_samples=500, n_features=10, beta_coef=[4, 3, 2, 2], epsilon=3, correlation_parameter=1, is_independent='independent', data_name='data3'), 'data3': data_generator(n_samples=500, n_features=10, beta_coef=[4, 3, 2, 2], epsilon=4, correlation_parameter=1, is_independent='independent', data_name='data4')}
            0         1         2         3         4         5         6  \
0    1.624345 -0.611756 -0.528172 -1.072969  0.865408 -2.301539  1.744812   
1    1.462108 -2.060141 -0.322417 -0.384054  1.133769 -1.099891 -0.172428   
2   -1.100619  1.144724  0.901591  0.502494  0.900856 -0.683728 -0.122890   
3   -0.691661 -0.396754 -0.687173 -

In [65]:
ga_resutls_dict = {}

for data_name, data in generated_data_dict.items():
    x, y = pd.DataFrame(data.data_gen()[0]), data.data_gen()[1]
    ga_best_score, ga_best_solution, ga_result_log = GA(x,y)
    new_name = data_name + '_with_' + 'GA'
    ga_resutls_dict[new_name] = ga_Results(ga_best_score, ga_best_solution, ga_result_log)

ValueError: Must pass 2-d input. shape=(30, 20, 10)

In [ ]:
ga_result_keys = list(ga_resutls_dict.keys())
ga_result_keys

In [ ]:
ga_reuslt_data = ga_resutls_dict[ga_result_keys[1]]

In [ ]:
ga_reuslt_data.best_score

In [ ]:
ga_reuslt_data.best_solution

In [ ]:
ga_reuslt_data.result_log[1]

### 최종결과 확인 

비고  
4개의 데이터를 input_paramer.xlcs 에서 생성해서 데이터를 만듬  
EXCEl 에서 수정을 통해 원하는데이터 개수만큼 생성 가능 

### SA

이때 sa 의경우는 실험 PARAMETER 4개를 추가로 입력받아서 결과를 출력  --> 16개 data
1개의 경우만을 이용해서 GA 와 비교한다면 1개의 열로 수정해서 진행 

### GA